In [67]:
#just need to extract the time for the LED pulse and produce a CSV with this in.
#LED square pulse should be going up to 2.9V but triggered when 960mV - find time on oscilloscope when this is true and the index of this

In [68]:
import sys
import time
import matplotlib.pyplot as plt 
import numpy as np 
import tekwfm
import os
from os.path import exists
from scipy import signal
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.special import gamma
import matplotlib.cm as cm
from collections import Counter

import gc
import re
import tempfile
from collections import defaultdict, namedtuple
from pathlib import Path
import pandas as pd

In [69]:
doVerbose = False

#import the wfm file
filename_base = "PMT5/270126_spe_2000_500_1_on_ch2"


#saving the processed file

folder_name = "PMT5/Proc_Clean_PMT5/CH2"
file_name = "270126_spe_2000_500_1_on_ch2_processed.csv" 
file_path = os.path.join(os.getcwd(), folder_name, file_name)


startEvent = 0
nEvents = 500


volts, tstart, tscale, tfrac, tdatefrac, tdate = tekwfm.read_wfm(filename_base+".wfm")
if doVerbose:
    print('>>>>', volts, tstart, tscale, tfrac, tdatefrac, tdate)

samples = volts.shape

print(len(volts))
print(tscale)
print('samples', samples[0])
tstop = samples[0]*tscale+tstart
sampleTimes = [tstart+x*tscale for x in range(samples[0])]

#already positive, don't need to flip like with CH1.
tempVolts = [volts[:, event] for event in range(startEvent, startEvent + nEvents)] # all events, array of array of voltages



12500
8e-11
samples 12500


In [70]:
def first_crossing_time(tempVolts, sampleTimes, threshold=0.960):

    for i in range(1, len(tempVolts)):
        if tempVolts[i-1] < threshold <= tempVolts[i]:
            #linear interpolation
            y1, y2 = tempVolts[i-1], tempVolts[i]
            x1, x2 = sampleTimes[i-1], sampleTimes[i]
    
            if (y2 - y1)==0:
                return None, None, None
            frac = (threshold - y1) / (y2 - y1)
            if not (0<=frac<=1):
                return None, None, None
        
            first_cross = x1 + frac * (x2 - x1)

            return first_cross
    
    #if no crossing found
    return None

def cross_index(first_cross, sampleTimes):
    if first_cross is None:
        return len(sampleTimes) - 1
    else:
        time_difference = np.abs(np.array(sampleTimes) - first_cross)
        cross_index = np.argmin(time_difference)
        return cross_index



In [71]:
#for all events
cross_times = []
cross_indices = []

for tempVolt in tempVolts:
    t_cross = first_crossing_time(tempVolt, sampleTimes)
    i_cross = cross_index(t_cross, sampleTimes)
    cross_times.append(t_cross)
    cross_indices.append(i_cross)

print(cross_times)
print(cross_indices)

[np.float64(-1.8168685714285715e-07), np.float64(-1.7714823529411766e-07), np.float64(-1.3569142857142858e-07), np.float64(-2.751726829268293e-07), np.float64(-4.080000000000021e-09), None, np.float64(-3.153169230769231e-07), np.float64(-2.4001641025641027e-07), np.float64(-3.809125925925926e-07), np.float64(-3.771314285714286e-07), np.float64(-5.7064000000000034e-08), np.float64(-1.1443833333333334e-07), np.float64(-1.5688e-07), np.float64(-3.605790476190476e-07), np.float64(-8.122000000000001e-08), np.float64(-2.6758666666666666e-07), np.float64(-2.3608275862068987e-08), None, np.float64(-1.2923891891891892e-07), None, np.float64(-6.195265306122449e-08), np.float64(-4.003840000000003e-08), np.float64(-1.0720000000000032e-08), np.float64(-1.5157694915254238e-07), np.float64(-6.413333333333335e-08), None, np.float64(-2.102646153846154e-07), np.float64(-3.8999999999999997e-07), np.float64(-1.0176888888888892e-07), None, np.float64(-1.7531e-07), np.float64(-3.1426e-07), np.float64(-1.032

In [72]:
#produce the csv file

df = pd.DataFrame({
            'crossing_trigger': cross_times,
            'crossing_index': cross_indices,
            })

df.to_csv(file_path, sep=',', encoding='utf-8-sig', index=True, header=True)

print('CSV done!')

CSV done!
